# CLIP (Contrastive Language-Image Pre-Training) Model Adapter Tutorial

This notebook provides a comprehensive guide on fine-tuning a CLIP model using the Dataloop platform and its Python SDK. CLIP models are powerful for tasks involving understanding the relationship between images and text, such as zero-shot image classification, image-text retrieval, and generating image embeddings for semantic search.

Fine-tuning allows you to adapt a pre-trained CLIP model to your specific dataset and domain, potentially improving its performance on tasks relevant to your data. This tutorial will walk you through:

1. Preparing a dataset with images and their corresponding textual descriptions.
2. Using the Dataloop CLIP model adapter to fine-tune the model on your prepared dataset.
3. Deploying the fine-tuned model and using it to generate embeddings for your images.

### Navigate through the following sections:
1. [Install Dependencies](#install-dependencies)
2. [Import Required Libraries](#import-libraries)
3. [Set Up Dataloop Environment](#setup-environment)
4. [Prepare Dataset for Fine-Tuning](#prepare-dataset)
    *   [4.1 Option A: Use Public Dataset (Mars Surface Images)](#use-public-dataset)
        *   [4.1.1 Install Mars Surface Images DPK](#install-mars-dpk)
        *   [4.1.2 Get Captioned Dataset and Split for ML](#get-mars-dataset-split)
    *   [4.2 Option B: (Alternative) Upload and Prepare Your Custom Dataset](#upload-custom-dataset)
5. [Fine-Tune and Deploy CLIP Model](#finetune-deploy-clip)
    *   [5.1 Install CLIP Model Package (DPK)](#install-clip-model-dpk)
    *   [5.2 Configure and Clone Pretrained CLIP Model](#configure-clone-clip-model)
    *   [5.3 Train the Model](#train-clip-model)
    *   [5.4 Deploy the Fine-Tuned Model](#deploy-clip-model)
    *   [5.5 Embed Datasets using the Fine-Tuned Model](#embed-datasets-clip)
6. [Conclusion](#conclusion)

Let's get started!

## <a id="install-dependencies"></a>1. Install Dependencies

First, ensure that the necessary Python libraries are installed. This notebook requires `dtlpy` for interacting with the Dataloop platform and `pandas` for data manipulation. The following cell will install or upgrade them.

In [ ]:
!pip install dtlpy pandas --upgrade --quiet

## <a id="import-libraries"></a>2. Import Required Libraries

Now, we import all the Python libraries that will be used throughout this tutorial.

[Back to Top](#top)

In [ ]:
import time
import random
import string
import dtlpy as dl
import pandas as pd

from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

## <a id="setup-environment"></a>3. Set Up Dataloop Environment

We need to set up our Dataloop environment and get our project. You'll need to replace project and dataset names with your own values.

> **_NOTE:_**  This tutorial assumes you are working in a new project which does NOT have the CLIP model previously installed. If it's an existing project and you already have CLIP installed, you will need to get the appropriate app and base CLIP model entity for the rest of the code to work correctly.

[Back to Top](#top)

In [ ]:
if dl.token_expired():
    dl.login()

# Define a unique name for your project or change it to your own project name
user_email = dl.info()['user_email']
user_prefix = user_email.split('@')[0].replace('.', '').replace('-', '') # Simple prefix from email
project_name = f'{user_prefix}-clip-tutorial'

# Check if the project exists, if not, create it
try:
    project = dl.projects.get(project_name=project_name)
    print(f"Successfully retrieved project: '{project.name}' (ID: {project.id})")
except dl.exceptions.NotFound:
    project = dl.projects.create(project_name=project_name)
    print(f"Successfully created project: '{project.name}' (ID: {project.id})")

**Action Required:** In the cell above, replace if you want `"{user_prefix}-clip-tutorial"` with the desired name for your Dataloop project. If a project with this name already exists, the SDK will retrieve it; otherwise, a new project will be created.

## <a id="prepare-dataset"></a>4. Prepare Dataset for Fine-Tuning

Fine-tuning a CLIP model requires a dataset of images paired with relevant textual descriptions. This section covers two ways to prepare such a dataset:
1.  **Option A:** Use a publicly available dataset from the Dataloop Marketplace (Mars Surface Images with Captions).
2.  **Option B:** Upload your own custom dataset of images and descriptions.

[Back to Top](#top)

### <a id="use-public-dataset"></a>4.1 Option A: Use Public Dataset (Mars Surface Images)

For this tutorial we will install the Mars Surface Images Datasets from the Dataloop Marketplace. This dataset includes images with descriptions pre-loaded in the item metadata.

#### <a id="install-mars-dpk"></a>4.1.1 Install Mars Surface Images DPK

This Dataloop Package (DPK) contains datasets related to Mars surface imagery, including one with captions.

In [ ]:
dpk = dl.dpks.get(dpk_name="mars-surface-images")
try:
    app = project.apps.install(
        app_name=dpk.display_name, 
        dpk=dpk, 
        custom_installation=dpk.to_json()
    )
    print(f"Installed {dpk.display_name} app: {app.name}")
except dl.exceptions.BadRequest as e:
    print(f"{dpk.display_name} app already installed, getting existing app")
    app = project.apps.get(app_name=dpk.display_name)

⚠️ You may need to wait a few minutes after installing the app until the dataset has completed loading into your project.

In [ ]:
# Wait for the dataset to load
time.sleep(150)

#### <a id="get-mars-dataset-split"></a>4.1.2 Get Captioned Dataset and Split for ML

After installing the DPK, the "Mars Surface Images with Captions" dataset should be available in your project. We will retrieve this dataset and split its items into training, validation, and test subsets. The Dataloop SDK provides a convenient method to do this by automatically tagging items. This splitting is crucial for proper model training and evaluation.

In [ ]:
dataset = project.datasets.get(dataset_name="Mars Surface Images with Captions")

SUBSET_PERCENTAGES = {'train': 80, 'validation': 10, 'test': 10}
dataset.split_ml_subsets(
        percentages=SUBSET_PERCENTAGES
    )

### <a id="upload-custom-dataset"></a>4.2 Option B: (Alternative) Upload and Prepare Your Custom Dataset

If you have your own dataset of images and corresponding text descriptions, you can use the function below to create a new Dataloop dataset and upload your images with descriptions. 

The function expects a Pandas DataFrame (`pairs_df`) with two columns:
-   `'filepath'`: The local path to each image file.
-   `'description'`: The text description for that image.

It also assumes that for each image file (e.g., `items/image1.jpg`), there is a corresponding JSON annotation file (e.g., `json/image1.json`) if you have existing annotations to upload. If not, the `local_annotations_path` part can be adapted.

[Back to Top](#top)

In [ ]:
def create_new_dataset(dataset_name, pairs_df, subset_percentages={'train': 80, 'validation': 10, 'test': 10}):
    """
    Creates a new dataset from a CSV file containing image paths and descriptions

    Args:
        dataset_name (str): Name of the dataset to create
        pairs_df (pd.DataFrame): DataFrame containing 'filepath' and 'img_description' columns
        subset_percentages (dict): Dictionary containing the percentages for each subset
        default is 80% train, 10% validation, 10% test
        can be changed to any other percentages as long as the sum is 100
    """

    try:
        dataset = project.datasets.create(dataset_name=dataset_name)
    except dl.exceptions.BadRequest:
        # Generate 5 random alphanumeric characters
        suffix = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
        dataset = project.datasets.create(dataset_name=f"{dataset_name}_{suffix}")

    def upload_item(row):
        file_path = row["filepath"]
        annots_path = file_path.replace("items", "json") # This assumes a specific structure for annotation files
        
        # Upload item with annotations
        item = dataset.items.upload(
            local_path=file_path,
            local_annotations_path=annots_path,
            item_metadata=dl.ExportMetadata.FROM_JSON,
            overwrite=True,
        )

        # Set description and update
        item.set_description(text=row["description"])
        item.update()

    # Use ThreadPoolExecutor to upload items in parallel with progress bar
    with ThreadPoolExecutor() as executor:
        from tqdm import tqdm
        list(tqdm(
            executor.map(upload_item, [row for _, row in pairs_df.iterrows()]),
            total=len(pairs_df),
            desc="Uploading items",
            unit="item",
            bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]'
        ))

    # Since model training requires labels, we create a dummy label for the recipe
    dataset.add_labels(label_list=['free-text'])
    # After uploading, you would also split this custom dataset similarly to Option A:
    # dataset.split_ml_subsets(percentages=subset_percentages)

    return dataset

## <a id="finetune-deploy-clip"></a>5. Fine-Tune and Deploy CLIP Model

With the dataset prepared, we can now proceed to fine-tune the CLIP model.

[Back to Top](#top)

### <a id="install-clip-model-dpk"></a>5.1 Install CLIP Model Package (DPK)

First, we install the CLIP model DPK from the Dataloop Marketplace. This package provides the necessary components for CLIP model training and inference.

In [ ]:
clip_model_dpk = dl.dpks.get(dpk_name="clip-model-pretrained")
try:
    model_app = project.apps.install(
        app_name=clip_model_dpk.display_name, 
        dpk=clip_model_dpk, 
        custom_installation=clip_model_dpk.to_json()
    )
    print(f"Installed {clip_model_dpk.display_name} app: {model_app.name}, ID: {model_app.id}")
except dl.exceptions.BadRequest as e:
    print(f"{clip_model_dpk.display_name} app already installed, getting existing app")
    model_app = project.apps.get(app_name=clip_model_dpk.display_name)

### <a id="configure-clone-clip-model"></a>5.2 Configure and Clone Pretrained CLIP Model

Next, we retrieve the base pre-trained CLIP model entity ("openai-clip") provided by the DPK. We'll then configure its metadata, specifying which subsets of our `dataset` to use for training and validation. 

You can also adjust model hyperparameters like learning rate, batch size, and number of epochs in the `base_model.configuration` dictionary. The example settings below are a starting point.

In [ ]:
base_model = project.models.get(model_name="openai-clip")

# Configure model metadata and subsets
# Use the correct field names for ML subset tags
train_filters = dl.Filters(field="metadata.system.tags.train", values=True)
val_filters = dl.Filters(field="metadata.system.tags.validation", values=True)

# Add subsets to the model using the correct method
base_model.add_subset(subset_name="train", subset_filter=train_filters)
base_model.add_subset(subset_name="validation", subset_filter=val_filters)

# Set model configuration (hyperparameters)
base_model.configuration = {
    "model_name": "ViT-B/32",      # CLIP model architecture (e.g., ViT-B/32, RN50)
    "embeddings_size": 512,       # Output embedding dimension for ViT-B/32
    "num_epochs": 10,             # Number of training epochs (adjust based on dataset size and convergence)
    "batch_size": 64,             # Batch size for training (adjust based on GPU memory)
    "learning_rate": 5e-6,        # Learning rate (often smaller for fine-tuning)
    "early_stopping": True,       # Enable early stopping
    "early_stopping_epochs": 3,   # Number of epochs with no improvement before stopping
    "weight_decay": 0.01          # Weight decay for regularization (optional)
}
base_model.output_type = "text" # For CLIP, this usually indicates it's working with text-image pairs

# Update the model to save the subset configurations
base_model.update(system_metadata=True)

### <a id="train-clip-model"></a>5.3 Train the Model

Now we clone the configured base model. This creates a new model entity in your project that will be fine-tuned. We associate our `dataset` (created in [Section 4.3](#convert-to-prompt-items)) with this new model and then start the training process.

> **NOTE**: The training process can take a significant amount of time, depending on your dataset size, model configuration, and the available compute resources (GPU type).

In [ ]:
# Ensure 'dataset' is defined from the conversion step
if 'dataset' not in locals() or dataset is None:
    raise ValueError("Error: 'dataset' is not defined. Please run section 4.3 to create it.")

finetuned_model_name = base_model.name + "-finetuned-" + ''.join(random.choices(string.ascii_lowercase + string.digits, k=4))
print(f"Cloning model to create: '{finetuned_model_name}' using dataset '{dataset.name}'")
finetuned_model = base_model.clone(model_name=finetuned_model_name, dataset=dataset)

print(f"Starting training for model: '{finetuned_model.name}' (ID: {finetuned_model.id}). This may take a while...")
execution = finetuned_model.train()
print(f"Training initiated. Execution ID: {execution.id}. You can monitor progress in the Dataloop platform.")

The cell below will periodically check the status of the training execution. You can also monitor the training progress, view logs, and see performance metrics directly in the Dataloop platform by navigating to your project, then Models, finding your `finetuned_model`, and checking its 'Executions' or 'Training' tabs.

In [ ]:
# Wait for training to complete
print(f"Waiting for training execution {execution.id} to complete...")
execution_status = dl.executions.get(execution_id=execution.id).get_latest_status().get('status')

while execution_status not in [dl.ExecutionStatus.SUCCESS, dl.ExecutionStatus.FAILED, dl.ExecutionStatus.CANCELED]:
    print(f"Training in progress (Status: {execution_status})... checking again in 5 minutes")
    time.sleep(300)  # Sleep for 5 minutes
    execution_status = dl.executions.get(execution_id=execution.id).get_latest_status().get('status') # Refresh execution status

    if execution_status == dl.ExecutionStatus.SUCCESS:
        print(f"Training completed successfully! Model ID: {finetuned_model.id}")
        # Update the local model object with the latest status and artifacts from the platform
        finetuned_model = project.models.get(model_id=finetuned_model.id)
    elif execution_status == dl.ExecutionStatus.FAILED:
        print(f"Training failed. Execution ID: {execution.id}. Check logs in Dataloop platform for details.")
    else:
        print(f"Training ended with status: {execution_status}. Execution ID: {execution.id}.")

### <a id="deploy-clip-model"></a>5.4 Deploy the Fine-Tuned Model

Once the model has successfully trained, you can deploy it as a service. This makes the model available for inference tasks, such as generating embeddings for new images.

[Back to Top](#top)

In [ ]:
if 'finetuned_model' in locals() and finetuned_model is not None and execution_status == dl.ExecutionStatus.SUCCESS:
    print(f"Deploying fine-tuned model: '{finetuned_model.name}' (ID: {finetuned_model.id})")
    # The service will be created with default settings (e.g., 1 replica, default GPU if needed by model)
    # You can customize deployment configuration if necessary via finetuned_model.deploy(service_config={...})
    service = finetuned_model.deploy()
    # Get the latest version of the model entity, which now includes deployment details
    finetuned_model = project.models.get(model_id=finetuned_model.id)
    while finetuned_model.status not in [dl.ModelStatus.DEPLOYED, dl.ModelStatus.FAILED]:
        print(f"Model '{finetuned_model.name}' is deploying in service {service.id}. Waiting for service to be ready...")
        time.sleep(180)
        finetuned_model = project.models.get(model_id=finetuned_model.id)
else:
    print("Skipping deployment: Model training was not successful or 'finetuned_model' is not defined.")

### <a id="embed-datasets-clip"></a>5.5 Embed Datasets using the Fine-Tuned Model

After the fine-tuned model is deployed and its service is ready, you can use it to generate embeddings for images in any dataset. These embeddings capture the semantic content of the images as understood by your fine-tuned model and can be used for tasks like semantic search or similarity comparison.

We will typically want to embed the original image dataset (e.g., "Mars Surface Images with Captions" or your custom image dataset), though you can also embed other datasets with your finetuned model.

[Back to Top](#top)

In [ ]:
if 'finetuned_model' in locals() and finetuned_model.status == dl.ModelStatus.DEPLOYED:
    # IMPORTANT: Select the dataset you want to embed. This should be your ORIGINAL image dataset,
    # not the prompt_dataset used for training.

    # You can embed the original image dataset or another dataset with the fine-tuned model.
    # For example, if you used the Mars dataset and want to embed the uncaptioned dataset (that the model was *not* trained on):
    dataset_to_embed = project.datasets.get(dataset_name="Mars Surface Images Unannotated") 
    # Or, if you used a custom dataset named 'MyCustomImageData' and want to embed it:
    # dataset_to_embed = project.datasets.get(dataset_name='MyCustomImageData')

    if dataset_to_embed:
        print(f"Starting embedding for dataset: '{dataset_to_embed.name}' (ID: {dataset_to_embed.id}) using model '{finetuned_model.name}'.")
        embedding_execution = finetuned_model.embed_datasets(dataset_ids=[dataset_to_embed.id])
        print(f"Embedding process initiated. Execution ID: {embedding_execution.id}. This may take some time.")
        # You can wait for this execution to complete similarly to the training execution if needed.
        # embedding_execution.wait() or a loop with time.sleep()
    else:
        print("Error: 'dataset_to_embed' is not defined. Please specify the correct original image dataset.")
else:
    print(f"Skipping embedding: Model {finetuned_model} is not successfully deployed.")

## <a id="conclusion"></a>6. Conclusion

Congratulations! You have successfully walked through the process of fine-tuning a CLIP model using the Dataloop platform. This included:

1.  **Setting up your Dataloop environment.**
2.  **Preparing a dataset** of images and descriptions, either by using a public dataset or your own custom data.
3.  **Installing the CLIP model package.**
4.  **Configuring and cloning** a pretrained CLIP model.
5.  **Training (fine-tuning)** the model on your dataset.
6.  **Deploying** the fine-tuned model as a service.
7.  **Generating embeddings** for an image dataset using your fine-tuned model.

### Next Steps
*   **Experiment with Hyperparameters:** Adjust learning rate, batch size, number of epochs, and CLIP model architecture (e.g., `ViT-L/14` if available and supported) to potentially improve performance.
*   **Evaluate Your Model:** While this tutorial focuses on the fine-tuning process, a crucial next step is to evaluate your fine-tuned model's performance on a held-out test set for tasks like image-text retrieval or zero-shot classification.
*   **Use Embeddings:** Explore applications of the generated embeddings, such as building a semantic image search engine or performing image clustering.
*   **Explore Advanced Features:** Dataloop offers many more features for MLOps, data management, and AI development. Check out the [Dataloop Developer Documentation](https://developers.dataloop.ai/) for more.